## Importing the libraries ##
- We going to use the numpy libraries to use numpy arrays 
- We going to use pandas to load, merge  and modify our dataset
- matplotlib and seaborn libraries are going to be used to plot the model 

In [5]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D


## Loading the dataset ##

- **Pandas** is used to load the datafiles into our workspace
- four datafiles were loaded named Riders, Test,Train as well as VariableDefinitions


In [8]:
dataset1 = pd.read_csv("./regression data/Riders.csv")
dataset2 = pd.read_csv("./regression data/Test.csv")
dataset3 = pd.read_csv("./regression data/Train.csv")
dataset4 = pd.read_csv("./regression data/VariableDefinitions.csv")

## Data Preprocessing ##
 - Some columns needs to dropped
 - train test and riders needs to be merged 
 - nulls values needs to be dealt with

## Encoding the categorical data ##

## check for Corellated predictors ( x variables) ##

## Feature Scaling ##

## Creating y and x metrics ##

## Spliting Data into the training and the test set ##

## Fitting the multivariate Regression model ##

## Assesing model accuracy ##